In [ ]:
# using Distributed
# nprocs() < 2 && addprocs(4);

In [ ]:
using Plots

In [ ]:
# using Plots
# using LinearAlgebra
# using Revise
using LatticeQM

# Tutorial: Tight-binding operators

In [ ]:
lat = Geometries.honeycomb_ABC()
H = Operators.graphene(lat; tz=0.3, ℓinter=0.08, ℓintra=0.04, cellrange=3, format=:dense)#, mode=:spinhalf)
# Materials.addhaldane!(hops, lat, 0.1; spinhalf=true, mode=:anti)
Operators.addinterlayerbias!(H, lat, 0.3)
# Operators.addzeeman!(H, lat, 1e-5)

# Operators.setfilling!(H, 0.5003; nk=400, T=0)

# Additional operators
# posZ = kron(Operators.positionalong(lat, [0,0,1.0]; rescale=true), Diagonal(ones(2)))
# sz = Operators.spin(lat, "sz")
# valley = Operators.valley(lat, x->x[5]; spinhalf=true)

posZ = Operators.positionalong(lat, [0,0,1.0]; rescale=true)
valley = Operators.valley(lat; spinhalf=false)
# valley = Operators.valley(lat, x->x[5]; spinhalf=false)

nothing

In [ ]:
plot(lat, "sublattice"; supercell=[3,3])

In [ ]:
ks = kpath(lat; num_points=500)
bands = getbands(H, ks, [valley, posZ])

Spectrum.getberry!(bands, H, ks) # add berry curvature to band data

plot(bands, 2; ylims=(-0.45,0.45), size=(1400,200),  colorbar=true)#, colorbar_title="spin")

In [ ]:
include("sampling.jl")
kgrid, kweights = getcustomsampling(; N=10, k=4, l=150)

# Calculate the density of states
energies = LinRange(-0.05,-0.27,500) |> collect
dos = Spectrum.getdos(H, energies, kgrid, kweights; format=:dense, Γ=0.005)

using DelimitedFiles
writedlm("output/energies.out", energies)
writedlm("output/dos.out", dos)
nothing

In [ ]:
lat = Geometries.honeycomb_ABC()
# lat = Lattices.superlattice(lat, [[1,1] [2,-1]])
# Lattices.foldPC!(lat)#; shift=[1/3,-1/3,0])

H = Operators.graphene(lat; tz=0.3, ℓinter=0.08, ℓintra=0.04, cellrange=2, format=:dense)#, mode=:spinhalf)
Operators.addinterlayerbias!(H, lat, 0.3)

posZ = Operators.positionalong(lat, [0,0,1.0]; rescale=true)
valley = Operators.valley(lat; spinhalf=false)
# valley = Operators.valley(lat, x->x[5]; spinhalf=false)

nothing

In [ ]:
include("sampling.jl")

kgrid, kweights = getcustomsampling(; N=10, k=4, l=70)
# kgrid = Structure.regulargrid(; nk=300^2)
Lattices.foldBZ!(kgrid, lat)
kpoints = Lattices.getB(lat) * kgrid

# # or for square grid in real space
# B = Lattices.getB(lat)[1:2,1:2]
# kpoints = B*[2/3,1/3] .+ 0.1*(Structure.regulargrid(; nk=200^2) .- [0.5,0.5])
# kgrid = inv(B)*kpoints

# Operators.setfilling!(H, 0.5; nk=9)
# bands = Spectrum.bandmatrix(H, kgrid; multimode=:distributed)
bands, obs = Spectrum.bandmatrix(H, kgrid, [valley, posZ]; multimode=:distributed2)

using DelimitedFiles

writedlm("output/kpoints.out", kpoints)
writedlm("output/bandmatrix.out", bands)
writedlm("output/obsmatrix.out", obs)
nothing

In [ ]:
energies = LinRange(-0.09,-0.27,20)

density = Spectrum.fermisurfacedensity_fromdata(bands, energies; broadening=0.004)
density_layer = Spectrum.fermisurfacedensity_fromdata(bands, energies, obs[:,:,2]; broadening=0.001)#; broadening=0.001)

using DelimitedFiles

writedlm("output/density.out", density)
writedlm("output/density_layer.out", density_layer)

nothing

In [ ]:
# uc_area = det(Lattices.basis(lat))*(2.46)^2 * 10^(-16) # in cm^2

In [ ]:
# scatter(kpoints[1,:], kpoints[2,:]; zcolor=vec(density[7,:]), markersize=1, markerstrokewidth=0, aspect_ratio=:equal)

In [ ]:
# scatter(kpoints[1,:], kpoints[2,:]; zcolor=vec(density_layer[10,:]), clims=(-1,1), color=:seismic, markersize=1, markerstrokewidth=0, aspect_ratio=:equal)

# Mean-field (no pairing allowed)

In [ ]:
using ProgressMeter; ProgressMeter.ijulia_behavior(:clear);

In [ ]:
using LinearAlgebra

lat = Geometries.honeycomb_ABC()
# lat = Lattices.superlattice(lat, [[1,1] [2,-1]])
# Lattices.foldPC!(lat)#; shift=[1/3,-1/3,0])

H = Operators.graphene(lat; tz=0.3, ℓinter=0.10, ℓintra=0.05, cellrange=3, format=:dense, mode=:spinhalf)
# Materials.addhaldane!(hops, lat, 0.1; spinhalf=true, mode=:anti)
Operators.addinterlayerbias!(H, lat, 0.3)

μ = -0.08 # energies[3]

filling = Spectrum.filling(H, μ; nk=100)
Operators.setfilling!(H, filling; )

sz = Operators.spin(lat, "sz")
valley = Operators.valley(lat; spinhalf=true)
# valley = Operators.valley(lat, x->x[5]; spinhalf=true)
posZ = kron(Operators.positionalong(lat, [0,0,1.0]; rescale=true), Diagonal(ones(2)))

v = Operators.getcappedyukawa(lat; format=:dense, spin=true, k0=0.4, U=3.5) # interaction potential
# v = Operators.gettanh(lat; format=:dense, spin=true, a=3.5, U=3.5)
ρ_init = Meanfield.initialguess(v, :random, :nonlocal; lat=lat) # initial guess
# ρ_init = Meanfield.initialguess(v, :ferro; lat=lat) # initial guess
nothing

In [ ]:
energies[4]

In [ ]:
# include("sampling.jl")
# kgrid, kweights = getcustomsampling(; N=10, k=4, l=150)

ρ_sol, ϵ_GS, HMF, converged, error = Meanfield.solvehartreefock( # run the calculation
    H, v, ρ_init, filling; klin=100, iterations=350, tol=1e-5,# p_norm=Inf,
    T=0.001, β=0.4,  show_trace=true, clear_trace=true
)

nothing

In [ ]:
ks = kpath(lat; num_points=1000)
bands_mf = getbands(HMF.h, ks, [sz, valley, posZ]; multimode=:distributed2)
bands_mf.bands .-= HMF.μ # shift chemical potential to zero

nothing

In [ ]:
plot(bands_mf, 1; ylims=(-1.5,1.5), size=(1400,200),  colorbar=true)
# plot(bands_mf, 1)

In [ ]:
bands = getbands(H, ks, [sz, valley, posZ]; multimode=:distributed2)

plot(bands, 3; ylims=(-0.95,0.95), size=(1400,200),  colorbar=true)
# plot(bands, 3)

In [ ]:
# bands_mf = getbands(HMF.h, ks, [sz, valley, posZ])
# bands_mf.bands .-= HMF.μ # shift chemical potential to zero

# # plot(bands_mf, 1; ylims=(-0.35,0.35), size=(800,200), colorbar=true)#, colorbar_title="spin")
# p1 = plot(bands, 3; ylims=(-1.35,1.35), size=(800,200), colorbar=true)#, colorbar_title="spin")
# p2 = plot(bands_mf, 3; ylims=(-1.35,1.35), size=(800,200), colorbar=true)#, colorbar_title="spin")

# plot(p1,p2)

# Mean-field with pairing allowed

In [ ]:
lat = Geometries.honeycomb_ABC()
H = Operators.graphene(lat; tz=0.25, ℓinter=0.15, ℓintra=0.07, format=:dense, mode=:spinhalf)
H = BdGOperator(H)
# Materials.addhaldane!(hops, lat, 0.1; spinhalf=true, mode=:anti)
Operators.addinterlayerbias!(H, lat, 0.3)

v = Operators.getcappedyukawa(lat; format=:dense, spin=true, k0=0.7, U=1.6) # interaction potential

ρ0_init = initialguess(v, :random, :nonlocal)
Δ0_init = initialguess(v, :random, :nonlocal)
ρ_init = BdGOperator(ρ0_init, Δ0_init) # initial guess

ρ_sol, ϵ_GS, HMF, converged, error = Meanfield.solvehartreefock( # run the calculation
    H, v, ρ_init, 0.5; klin=50, iterations=800, tol=1e-5,# p_norm=Inf,
    T=0.01, β=0.7,  show_trace=true, clear_trace=true
)
Operators.addchemicalpotential!(HMF.h, -HMF.μ)

nothing

In [ ]:
eOP = Superconductivity.electron(HMF.h)
bands_mf = getbands(HMF.h, ks, eOP.h)

# plot(bands_mf, 1; ylims=(-0.35,0.35), size=(800,200), colorbar=true)#, colorbar_title="spin")
p1 = plot(bands, 1; ylims=(-0.35,0.35), size=(800,200), colorbar=true)#, colorbar_title="spin")
p2 = plot(bands_mf, 1; ylims=(-0.35,0.35), size=(800,200), colorbar=true)#, colorbar_title="spin")

plot(p1,p2)

# Magnetic field, Hofstadter butterfly

In [ ]:
lat = Geometries.honeycomb_ABC()
H = Operators.graphene(lat; tz=0.45, ℓinter=0.01, ℓintra=0.01, format=:dense, mode=:nospin)
# Materials.addhaldane!(hops, lat, 0.1; spinhalf=true, mode=:anti)
# Operators.addinterlayerbias!(H, lat, 0.3)

fluxes, energies = Operators.hofstadter(H, lat, 50);
nothing

In [ ]:
p = plot()
for (ϕ,Es)=zip(fluxes,energies)
    scatter!(p, repeat([ϕ],length(Es)), Es; markersize=0.75, markercolor=:black, legend=false)
end
plot!(p, xlabel="flux per unit cell", ylabel="Energy E/t")
plot!(p, size=(400,400))
p